In [1]:
from scipy import ndimage, misc
import numpy as np
import os
import pandas as pd
from pandas import HDFStore, DataFrame

import h5py

from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Convolution2D, MaxPooling2D, ZeroPadding2D
from keras.utils.np_utils import to_categorical
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD

Using Theano backend.
Using gpu device 0: GeForce GTX 1070 (CNMeM is enabled with initial size: 80.0% of memory, cuDNN 5005)


In [12]:
dataset_dir = ('../dataset/')

In [13]:
store = HDFStore(os.path.join(dataset_dir,'labels.h5'))
ava_table = store['labels_train']
Y_test =  to_categorical(store['labels_test'].ix[:,'good'].as_matrix(),2)


In [14]:
h5f = h5py.File(os.path.join(dataset_dir,'images.h5'),'r')
X = h5f['data_test'][:]
X_test = np.hstack(X).reshape(19924,-1)

In [15]:
X_train = np.hstack(h5f['data']).reshape(100000,-1) 
Y_train = ava_table.ix[:, "good"].as_matrix()
Y_train = to_categorical(Y_train, 2)

Y_train = Y_train[:100000]

In [16]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train -= np.mean(X_train)
X_test -= np.mean(X_test)

X_train /= 255
X_test /= 255

In [17]:
model = Sequential()
model.add(Dense(64, input_dim=12288, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [20]:
model.fit(X_train, Y_train, nb_epoch=20, batch_size=64)

Epoch 1/20
100000/100000 [==============================] - 3s - loss: 0.6334 - acc: 0.7152     
Epoch 2/20
100000/100000 [==============================] - 3s - loss: 0.5827 - acc: 0.7326     
Epoch 3/20
100000/100000 [==============================] - 3s - loss: 0.5818 - acc: 0.7325     
Epoch 4/20
100000/100000 [==============================] - 3s - loss: 0.5813 - acc: 0.7325     
Epoch 5/20
100000/100000 [==============================] - 3s - loss: 0.5812 - acc: 0.7325     
Epoch 6/20
100000/100000 [==============================] - 3s - loss: 0.5811 - acc: 0.7325     
Epoch 7/20
100000/100000 [==============================] - 3s - loss: 0.5813 - acc: 0.7325     
Epoch 8/20
100000/100000 [==============================] - 3s - loss: 0.5813 - acc: 0.7325     
Epoch 9/20
100000/100000 [==============================] - 3s - loss: 0.5811 - acc: 0.7325     
Epoch 10/20
100000/100000 [==============================] - 3s - loss: 0.5812 - acc: 0.7325     
Epoch 11/20
100000/100000 [==

In [24]:
score = model.evaluate(X_test, Y_test)

print()
print('Test score:', score[0])
print('Test accuracy:', score[1])

19904/19924 [============================>.] - ETA: 0s
Test score: 0.762075347897
Test accuracy: 0.55154587431


In [29]:
#from keras.utils.visualize_util import plot
#plot(model, to_file='model.png')